In [ ]:
pip install shap

In [13]:
import warnings

warnings.filterwarnings("ignore")

In [14]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
#import h2o
#from h2o.automl import H2OAutoML
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import shap


In [15]:
df_main_1 = pd.read_csv('D:/OneDrive - Diacto Technologies Private Limited/Documents/DS/Regression/Datasets/train _house.csv')


In [16]:
df_main_1

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [17]:
df_main_copy = df_main_1.copy()

In [18]:

for col in df_main_1.columns:
    if df_main_1[col].dtype == object:
        try:
            df_main_1[col] = pd.to_numeric(df_main_1[col])
        except ValueError:
            try:
                df_main_1[col] = pd.to_datetime(df_main_1[col])
            except ValueError:
                pass
def null_handling(df_main, null_method = None, columns_name = None,):
  if null_method == 'Deletion':
    df_main = df_main.drop(columns = columns_name)

  elif null_method == 'Mean Imputing':
    imputer = SimpleImputer(strategy='mean')
    df_main = imputer.fit_transform(df_main[columns_name])

  elif null_method == 'Mode Imputing':
    imputer = SimpleImputer(strategy='most_frequent')
    df_main = imputer.fit_transform(df_main[columns_name])

  elif null_method == 'Median Imputing' :
    imputer = SimpleImputer(strategy='median')
    df_main = imputer.fit_transform(df_main[columns_name])

  else:
    df_main = df_main.copy()

  return df_main

def encoding(df_main, encoding_method = None, categorical_columns = None):
  #categorical_columns = df_main.select_dtypes(include=['object']).columns.tolist()
  if encoding_method == 'Delete':
    df = df_main.drop(columns = categorical_columns)

  elif encoding_method == 'Label Encoding':
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    label_encoder = LabelEncoder()
    for column in categorical_columns:
      df_main_copy[column] = label_encoder.fit_transform(df_main_copy[column])
    #df = df_main_copy.drop(columns=categorical_columns)
    df = df_main_copy

  elif encoding_method == 'Create Dummies':
    print('#################')
    dummy_df = pd.get_dummies(df_main[categorical_columns], drop_first=True)
    # Concatenate the dummy variables with the original DataFrame
    df = pd.concat([df_main.drop(columns=categorical_columns), dummy_df], axis=1)

  else:
    df = df_main.copy()

  return df

numerical_columns = df_main_1.select_dtypes(include = ['int64','float64']).columns.tolist()
df_main_num = null_handling(df_main_1,'Mean Imputing', numerical_columns)
df_main_num = pd.DataFrame(df_main_num, columns=numerical_columns)  # Convert to DataFrame


categorical_columns_null = df_main_1.select_dtypes(include = ['object']).columns.tolist()
df_main_cat = null_handling(df_main_1,'Mode Imputing', categorical_columns_null)
df_main_cat = pd.DataFrame(df_main_cat, columns=categorical_columns_null)  # Convert to DataFrame


df_main = pd.concat([df_main_num, df_main_cat], axis=1)

categorical_columns = df_main.select_dtypes(include=['object']).columns.tolist()
df = encoding(df_main,'Create Dummies',categorical_columns )



#################


In [19]:
y = df['SalePrice']
X = df.drop(['SalePrice'], axis=1)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)


In [21]:

model = RandomForestRegressor(random_state = 42)

model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [22]:
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE):", rmse)

Root Mean Squared Error (RMSE): 31111.748876265126


In [23]:
explainer = shap.Explainer(model) # model ran here is randomforest
shap_values = explainer(X_test)
shap_values_df = pd.DataFrame(data=shap_values.values, columns=original_column_names)
shap_values_df

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,247.571169,206.918413,-5530.926948,8016.701742,142063.091519,-142.293231,967.372480,1244.792222,-680.980001,15781.258290,...,12.335344,0.634229,-32.381695,0.143617,-120.399641,-0.116383,-1.967795,20.240197,-166.368199,-319.341333
1,74.799855,853.897621,991.070711,1422.209916,68566.051563,402.728092,983.366621,893.276363,-1266.616239,-4530.623399,...,2.588617,0.334085,-69.402297,0.239420,-45.196708,-0.080354,-1.869790,23.645145,-72.530792,-56.046468
2,79.319258,272.745062,-1119.717645,-887.218529,13870.400446,-40.252700,-147.878588,-3052.035401,-113.229907,2857.872226,...,2.075177,0.440497,-22.074480,0.082500,-9.359475,-0.157628,-7.114748,48.568708,-74.799351,-35.099083
3,-348.969878,22.970245,-768.041439,-1784.222085,-28720.676439,1153.305636,-2242.228644,2053.757444,-340.053399,-682.835369,...,0.670322,0.815015,-20.153513,0.393170,8.720200,-0.884662,-1.342826,20.898438,85.539809,-23.197516
4,-93.054848,116.026014,61.212598,944.297281,5901.122355,-85.550848,2511.002422,1095.537411,-216.708926,4043.088553,...,0.958497,1.487657,-161.153671,0.139786,6.414166,-0.145254,-1.657895,108.700436,-67.385809,-59.078967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,-98.891821,306.438003,-37.774974,-1681.682348,-20675.656743,-93.366675,4264.370506,1929.415747,-239.037168,723.079252,...,1.099754,0.355412,-30.585318,0.414221,9.226901,-0.161582,-2.579545,47.482261,52.541673,-21.078626
288,-239.559528,-327.961031,-1475.370840,-3430.519161,-27147.204168,-558.508755,811.902292,-1005.920287,809.309442,-1657.353436,...,2.034215,-0.472731,-14.961731,0.448366,-1.994327,-0.295664,-2.245630,20.234627,231.810683,-15.110296
289,183.850789,53.009631,-531.056973,-115.631099,-29294.810032,463.843682,1181.558496,-1684.353578,-309.161359,1119.389202,...,-24.519944,-0.062553,-17.417248,1.393894,12.702343,-0.241516,-1.888588,25.830353,73.816704,-16.453213
290,83.589594,558.000914,324.820354,-1296.443378,10539.576368,922.465599,1902.086841,1175.754210,-302.292457,-1258.993726,...,1.104688,1.025711,-56.240257,0.233118,-16.515814,-0.131102,-1.772203,88.014626,7.107022,-36.950451


In [28]:
def shap_function(model = None, X_test = None):
    explainer = shap.Explainer(model) # model ran here is randomforest
    shap_values = explainer(X_test)
    
    original_column_names = X_test.columns
    shap_values_df = pd.DataFrame(data=shap_values.values, columns=original_column_names)
    return shap_values_df

In [29]:
shap_values_dff = shap_function(model,X_test)
shap_values_dff

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,247.571169,206.918413,-5530.926948,8016.701742,142063.091519,-142.293231,967.372480,1244.792222,-680.980001,15781.258290,...,12.335344,0.634229,-32.381695,0.143617,-120.399641,-0.116383,-1.967795,20.240197,-166.368199,-319.341333
1,74.799855,853.897621,991.070711,1422.209916,68566.051563,402.728092,983.366621,893.276363,-1266.616239,-4530.623399,...,2.588617,0.334085,-69.402297,0.239420,-45.196708,-0.080354,-1.869790,23.645145,-72.530792,-56.046468
2,79.319258,272.745062,-1119.717645,-887.218529,13870.400446,-40.252700,-147.878588,-3052.035401,-113.229907,2857.872226,...,2.075177,0.440497,-22.074480,0.082500,-9.359475,-0.157628,-7.114748,48.568708,-74.799351,-35.099083
3,-348.969878,22.970245,-768.041439,-1784.222085,-28720.676439,1153.305636,-2242.228644,2053.757444,-340.053399,-682.835369,...,0.670322,0.815015,-20.153513,0.393170,8.720200,-0.884662,-1.342826,20.898438,85.539809,-23.197516
4,-93.054848,116.026014,61.212598,944.297281,5901.122355,-85.550848,2511.002422,1095.537411,-216.708926,4043.088553,...,0.958497,1.487657,-161.153671,0.139786,6.414166,-0.145254,-1.657895,108.700436,-67.385809,-59.078967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,-98.891821,306.438003,-37.774974,-1681.682348,-20675.656743,-93.366675,4264.370506,1929.415747,-239.037168,723.079252,...,1.099754,0.355412,-30.585318,0.414221,9.226901,-0.161582,-2.579545,47.482261,52.541673,-21.078626
288,-239.559528,-327.961031,-1475.370840,-3430.519161,-27147.204168,-558.508755,811.902292,-1005.920287,809.309442,-1657.353436,...,2.034215,-0.472731,-14.961731,0.448366,-1.994327,-0.295664,-2.245630,20.234627,231.810683,-15.110296
289,183.850789,53.009631,-531.056973,-115.631099,-29294.810032,463.843682,1181.558496,-1684.353578,-309.161359,1119.389202,...,-24.519944,-0.062553,-17.417248,1.393894,12.702343,-0.241516,-1.888588,25.830353,73.816704,-16.453213
290,83.589594,558.000914,324.820354,-1296.443378,10539.576368,922.465599,1902.086841,1175.754210,-302.292457,-1258.993726,...,1.104688,1.025711,-56.240257,0.233118,-16.515814,-0.131102,-1.772203,88.014626,7.107022,-36.950451
